In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
import struct
from mlxtend.data import loadlocal_mnist
import matplotlib.pyplot as plt

In [2]:
class CustomDataset(Dataset):
    def __init__(self, data_path, label_path):
        self.data, self.labels = self.load_data(data_path, label_path)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
    def load_data(self, data_path, label_path):
        return loadlocal_mnist(images_path=data_path, labels_path=label_path)

In [3]:
# train-images-idx3-ubyte: training set images (9912422 bytes)
# train-labels-idx1-ubyte: training set labels (28881 bytes)
# t10k-images-idx3-ubyte: test set images (1648877 bytes)
# t10k-labels-idx1-ubyte: test set labels (4542 bytes)

In [4]:
train_image_path = "../data/t10k-images.idx3-ubyte"
train_label_path = "../data/t10k-labels.idx1-ubyte"

In [5]:
# Retrieve Dataset
dataset = CustomDataset(train_image_path, train_label_path)

In [7]:
print(dataset[0][1])

7
